In [31]:
import numpy as np
import pandas as pd
import json

In [32]:
annotations = r"\\shelter\Kyu\unstain2stain\tiles\registered_stitched\OTS_14684_3_he.geojson"

In [33]:
with open(annotations) as f:
      features = json.load(f)["features"]

In [34]:
contour_df = pd.DataFrame({'contour':[np.squeeze(_['geometry']['coordinates']) for _ in features]})

C:\Users\Kevin\.conda\envs\wsi_analysis1\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [35]:
classlabel_list = []
for idx in range(len(features)):
      contour = features[idx]
      classlabel = contour["properties"]["classification"]["name"]
      classlabel_list.append(classlabel)

In [36]:
label_df = pd.DataFrame(classlabel_list,columns=["label"])

In [38]:
final_df = pd.concat([contour_df,label_df],axis=1)

In [39]:
final_df

,contour,label
0,"[[38297.0, 67349.0], [38295.0, 67350.0], [3829...",corneum
1,"[[38070.0, 67032.0], [38056.0, 67033.0], [3804...",spinosum
2,"[[38305.5, 67369.0], [38304.0, 67370.0], [3830...",white
3,"[[38023.0, 66903.0], [38014.0, 66904.0], [3800...",ecm
4,"[[38089.0, 58716.0], [38089.0, 58719.0], [3809...",hairfollicle
...,...,...
98,"[[46352, 66260], [46350, 66262], [46348, 66262...",spinosum
99,"[[67227, 54067], [67227, 54069], [67225, 54069...",sweat
100,"[[58940, 56662], [58940, 56663], [58939, 56663...",sweat
101,"[[19400.0, 55087.0], [19400.0, 55091.0], [1939...",white
